<a href="https://colab.research.google.com/github/LikeABug/class_RSU/blob/main/modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Programa de entrenamiento red neuronal **Mobile NET** para clasificacion de imagenes de residuos solidos urbanos de 4 clases.


---

*   PAPEL/CARTON
*   PLASTICO
*   METAL
*   VIDRIO






In [ ]:
#IMPORTACION DE LIBRERIAS

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import np_utils
import os
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

Preprosesamiento y preparacion de imagenes

In [ ]:
#DEFINICION DE VARIABLES
width_shape = 224           #ancho de imagen
height_shape = 224          #largo de imagen
num_classes = 4             #4 clases a clasificar (papel, vidrio, metal, plastico)
epochs = 20                 #Numero de epocas <<<----- Variable
batch_size = 32             #tamaño de lote

In [ ]:
#directorios de acceso a imagenes de entrenamiento y validacion
train_data_dir = '/gdrive/My Drive/224pixel/prueba/entrenamiento'  
validation_data_dir = '/gdrive/My Drive/224pixel/prueba/validacion'

In [ ]:
train_datagen = ImageDataGenerator(                   #generador de imagenes de entrenamiento
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

valid_datagen = ImageDataGenerator(                   #generador de imagenes de validacion 
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    preprocessing_function=preprocess_input)

#las imagenes se hacen pasar por los generadores y se guardan en su respectiva variable
#esto es lo que se da a la red neuronal como entrada en el entrenamiento
train_generator = train_datagen.flow_from_directory(  
    train_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #save_to_dir='',
    class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(  
    validation_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #save_to_dir='',
    class_mode='categorical')

#lo que se observa abajo es un recuento del total de imagenes de entrenamiento(arriba) y validacion(abajo)

Found 27314 images belonging to 4 classes.
Found 11707 images belonging to 4 classes.


In [ ]:
#para el calculo de los pasos por epoca se necesita el valor total de las imagenes de entrenamiento
#y validacion, por simplicidad solo ponemos en estas variables un valor menor al del total en cada
#caso

nb_train_samples = 10000
nb_validation_samples = 5000

In [ ]:
from keras.applications.mobilenet import MobileNet          

image_input = Input(shape=(width_shape, height_shape, 3))                               #definicion de la forma de la imagen de entrada
m_MobileNet = MobileNet(input_tensor=image_input, include_top=False,weights='imagenet') #se guarda el modelo sin la ultima capa
m_MobileNet.summary() #se imprime el modelo original

#Se agregan dos capas densas al modelo + la capa de salida de nuestras clases
last_layer = m_MobileNet.layers[-1].output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu', name='fc2')(x)
x=Dropout(0.3)(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()  #se imprime el modelo modificado

custom_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy']) #Se compila el modelo con funcion de perdida
                                                                                                #categorical_crossentropy, optimizador adadelta y
                                                                                                #metrica a mejorar accuracy

17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________

**ENTRENAMIENTO**

In [ ]:
#Ruta para guardar avances en el entramiento
filepath="/gdrive/My Drive/224pixel/checkpoints"

In [ ]:
#se carga el ultimo modelo en filepath, si no hay modelo se carga el custom_model porque significa que es el primer entrenamiento
from keras.models import Sequential, load_model
from numpy.testing import assert_allclose
 
try:
  new_model = load_model(filepath)
except OSError:
  new_model = custom_model

'assert_allclose(custom_model.predict(train_generator),\n                new_model.predict(train_generator),\n                1e-5)'

In [ ]:
# Entrenamiento del modelo
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model_history = new_model.fit(  
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size,
    callbacks=callbacks_list)

Epoch 1/20
312/312 [==============================] - ETA: 0s - loss: 0.9344 - accuracy: 0.6146
Epoch 00001: loss improved from inf to 0.93440, saving model to /gdrive/My Drive/224pixel/checkpoints
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /gdrive/My Drive/224pixel/checkpoints/assets
312/312 [==============================] - 3309s 11s/step - loss: 0.9344 - accuracy: 0.6146 - val_loss: 0.7474 - val_accuracy: 0.6973
Epoch 2/20
312/312 [==============================] - ETA: 0s - loss: 0.8895 - accuracy: 0.6313
Epoch 00002: loss improved from 0.93440 to 0.88946, saving model to /gdrive/My Drive/224pixel/checkpoints
INFO:tensorflow:Assets written to: /gdrive/My Drive/224pixel/checkpoints/assets
312/312 [==============================] - 2440s 8s/step - loss: 0.8895 

In [ ]:
new_model.save("/gdrive/My Drive/224pixel/prueba/model_Mobilenet.h5")  #Se guarda el modelo final ENTRENADO
